# Generative Models Classification Accuracy - MNIST
* Using a sample of MNIST test set - 600 imgs

**Future**
* No optimization of KLD weighting
* No attempt to start with a good initialization
* Using 50 iterations and 50 restarts for each image

In [9]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from utils.loading import load_net
from utils.data import make_generators_DF_MNIST
from utils.display import torch_to_np, show_imgs
from models.generative_classify import optimize_latent_cvae, gen_classify_cvae, optimize_latent, gen_classify

import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torch.nn import functional as F
import torch.optim as optim

import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

cuda:1
0


## CVAE
* Switch to normal batches???

In [2]:
def eval_gen_cvae(dataloader, model, labels, device, num_times=50, iterations=50, latent_size=16):
    model.eval()
    all_results = pd.DataFrame()
    
    with torch.no_grad():
        for i, (tensor_img, label, path) in enumerate(tqdm(dataloader)):
            path = path[0]
            
            tensor_img = tensor_img.to(device)
            results, predicted_label = gen_classify_cvae(tensor_img, labels, model, num_times=num_times, 
                                                         iterations=iterations, latent_size=latent_size, 
                                                         device=device, KLD_weight=1)
                        
            for i, true_label in enumerate(label): 
                all_results = all_results.append({'path': path, 
                                                  'true_label': int(true_label.cpu().numpy()),
                                                  'predicted_label': int(predicted_label), 
                                                 }, ignore_index=True)
    return all_results

In [12]:
model_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/models/CVAE-1_16_32_64-16-MNIST-10_model_best.pth.tar'
model = load_net(model_loc).to(device).eval()

files_df_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/files_df.pkl'
with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
files_df['val'] = files_df['val'].sample(n=100)

dataloaders = make_generators_DF_MNIST(files_df, batch_size=1, num_workers=2, size=32,
                                       path_colname='path', adv_path_colname=None, return_loc=True, bw=True)

labels = list(range(10))

with torch.cuda.device(1):
    results = eval_gen_cvae(dataloaders['val'], model, labels, device, num_times=50, iterations=50, latent_size=16)
    
SAVE_PATH = '/media/rene/data/adv_gen/MNIST/mnist_normal/models'
pickle.dump(results, open(str(SAVE_PATH)+'/'+'CVAE-1_16_32_64-16-MNIST-10_model_best'+'_100_'+'results.pkl', "wb"))

In [11]:
acc = len(results[results['predicted_label']==results['true_label']])/len(results)
acc

0.88

In [6]:
def eval_gen_vae(dataloader, model_dict, device, num_times=50, iterations=50, latent_size=16):
    all_results = pd.DataFrame()
    
    with torch.no_grad():
        for i, (tensor_img, label, path) in enumerate(tqdm(dataloader)):
            path = path[0]
            
            tensor_img = tensor_img.to(device)
            results, predicted_label = gen_classify(tensor_img, model_dict,
                                          num_times=50, iterations=50,
                                          latent_size=16, device=device, KLD_weight=1)
            
            
                        
            for i, true_label in enumerate(label): 
                all_results = all_results.append({'path': path, 
                                                  'true_label': int(true_label.cpu().numpy()),
                                                  'predicted_label': int(predicted_label), 
                                                 }, ignore_index=True)
    return all_results

In [ ]:
save_path = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/models/')
model_dict = {}
all_recon = []

for label in range(10):
    model_name = 'VAE-1_16_32_64-16-MNIST_label_'+str(label)+'_model_best.pth.tar'
    model_loc = save_path / model_name
    model_dict[label] = load_net(model_loc).to(device).eval()
    
files_df_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/files_df.pkl'
with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
files_df['val'] = files_df['val'].sample(n=100)

dataloaders = make_generators_DF_MNIST(files_df, batch_size=1, num_workers=2, size=32,
                                       path_colname='path', adv_path_colname=None, return_loc=True, bw=True)

with torch.cuda.device(1):
    results = eval_gen_vae(dataloaders['val'], model_dict, device, num_times=50, iterations=50, latent_size=16)

    
SAVE_PATH = '/media/rene/data/adv_gen/MNIST/mnist_normal/models'
pickle.dump(results, open(str(SAVE_PATH)+'/'+'VAE-1_16_32_64-16-MNIST-10_model_best'+'_100_'+'results.pkl', "wb"))

 32%|███▏      | 32/100 [13:31<28:44, 25.36s/it]

In [10]:
file = open("/media/rene/data/adv_gen/MNIST/mnist_normal/models/VAE-1_16_32_64-16-MNIST-10_model_best_100_results.pkl",'rb')
results = pickle.load(file)

acc = len(results[results['predicted_label']==results['true_label']])/len(results)
acc

0.85

In [ ]:
Test a normal VAE too of MNIST for accuracy using this -- Should be a script???
---Ask the weird variable thing on pytorch form if needed
git add
Figure out how to do proper adversarial attacks.
Review stuff?